In [2]:
# Import necessary libraries
import os
import pandas as pd

# Define the directory where the CSV files are located
input_dir_path = "data/sensor_data/"

# Define the directory where to save output files
output_dir_path = "data/dailydata/"

# Create an empty list to store the dataframes
dfs = []

# Loop over every file in the directory
for file_name in os.listdir(input_dir_path):
    # Check if the file is a CSV file
    if not file_name.endswith('mobil.csv') and file_name.endswith('.csv') and file_name.startswith('patch'):
        # Construct the full file path
        input_file_path = os.path.join(input_dir_path, file_name)

        # Load the data
        df = pd.read_csv(input_file_path,  na_values="NA")

        # Combine 'Date' and 'Time' into a single datetime column
        df['datetime'] = pd.to_datetime(df['dateTime'])

        # Drop the original 'Date' and 'Time' columns
        df.drop(['dateTime'], axis=1, inplace=True)

        # Set 'datetime' as the index
        df.set_index('datetime', inplace=True)

        # Resample the data to daily frequency, taking the mean of each day
        df_daily = df.resample('D').mean()
        
        # Get patch name and add to df_daily
        patch = file_name.split('_')[1].split('.')[0]
        df_daily.insert(0, 'patch', patch)

        # Add the resampled dataframe to the list
        dfs.append(df_daily)

        # Create the output file path
        output_file_path = os.path.join(output_dir_path, file_name.replace('.csv', '_daily.csv'))

        # Save the resampled dataframe to a new CSV file
        df_daily.to_csv(output_file_path, sep=";")

# Concatenate all the dataframes in the list
df_all = pd.concat(dfs).sort_index()

# Create the output file path for the final dataframe
output_all_file_path = os.path.join(output_dir_path, "all_data_daily.csv")

# Save the final dataframe to a CSV file
df_all.to_csv(output_all_file_path, sep=";")
